In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [16]:
df = pd.read_csv("../datasets/covid_toy.csv")
df.sample()

,age,gender,fever,cough,city,has_covid
13,64,Male,102.0,Mild,Bangalore,Yes


In [17]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

In [18]:
df.isna().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [19]:
df["cough"].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [20]:
from sklearn.model_selection import train_test_split

X_train , X_test, y_train, y_test = train_test_split(
    df.drop(columns=["has_covid"]),
    df["has_covid"],
    train_size=0.7,
    random_state=42
)

### Without Column transformer

In [21]:
si = SimpleImputer()

X_train_fever = si.fit_transform(X_train[["fever"]])
X_test_fever = si.transform(X_test[["fever"]])

X_train_fever.shape

(70, 1)

In [24]:
oe = OrdinalEncoder(categories=[["Mild", "Strong"]])

X_train_cough = oe.fit_transform(X_train[["cough"]])
X_test_cough = oe.transform(X_test[["cough"]])

X_train_cough.shape

(70, 1)

In [30]:
ohe = OneHotEncoder(drop="first", sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[["gender", "city"]])
X_test_gender_city = ohe.transform(X_test[["gender", "city"]])

In [31]:
X_train_gender_city.shape

(70, 4)

In [56]:
X_train_age = X_train.drop(columns=["gender", "fever", "cough", "city"]).values
X_test_age = X_test.drop(columns=["gender", "fever", "cough", "city"]).values
X_train_age.shape

(70, 1)

In [59]:
X_train_tr = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)
X_test_tr = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)

In [62]:
X_train_tr.shape

(70, 7)

### Column Tranformer

In [63]:
from sklearn.compose import ColumnTransformer

In [64]:
transformer = ColumnTransformer(transformers=[
    # Name            # Transformer Class                               # Column to be applied on
    ("SimpleImputer", SimpleImputer()                                 , ["fever"]),
    ("Nominal"      , OneHotEncoder(drop="first", sparse_output=False), ["gender", "city"]),
    ("Ordinal"      , OrdinalEncoder(categories=[["Mild", "Strong"]]) , ["cough"]),
], remainder ="passthrough" )

In [69]:
X_train_tr = transformer.fit_transform(X_train)
X_train_tr.shape

(70, 7)

In [70]:
X_test_tr = transformer.transform(X_test)

In [71]:
X_test_tr.shape

(30, 7)